# Applying data minimization with categorical data and only a subset of the features to a trained ML model

In this tutorial we will show how to perform data minimization for ML models using the minimization module.

This will be demonstarted using the German Credit dataset (original dataset can be found here: https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data).

## Load data
QI parameter determines which features will be minimized.

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

import sys
sys.path.append('/Users/leodom01/Repos/ai-privacy-DataProtectionTechonolgies')

import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from apt.utils.dataset_utils import get_german_credit_dataset_pd

(x_train, y_train), (x_test, y_test) = get_german_credit_dataset_pd()
features = ["Existing_checking_account", "Duration_in_month", "Credit_history", "Purpose", "Credit_amount",
                "Savings_account", "Present_employment_since", "Installment_rate", "Personal_status_sex", "debtors",
                "Present_residence", "Property", "Age", "Other_installment_plans", "Housing",
                "Number_of_existing_credits", "Job", "N_people_being_liable_provide_maintenance", "Telephone",
                "Foreign_worker"]
categorical_features = ["Existing_checking_account", "Credit_history", "Purpose", "Savings_account",
                        "Present_employment_since", "Personal_status_sex", "debtors", "Property",
                        "Other_installment_plans", "Housing", "Job"]
QI = ["Duration_in_month", "Credit_history", "Purpose", "debtors", "Property", "Other_installment_plans",
      "Housing", "Job"]

print(x_train)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/german'

## Train decision tree model
we use OneHotEncoder to handle categorical features.

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
numeric_features = [f for f in features if f not in categorical_features]
numeric_transformer = Pipeline(
    steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0))]
)
categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse=False)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)
encoded_train = preprocessor.fit_transform(x_train)
model = DecisionTreeClassifier()
model.fit(encoded_train, y_train)

encoded_test = preprocessor.transform(x_test)
print('Base model accuracy: ', model.score(encoded_test, y_test))

Base model accuracy:  0.6933333333333334


## Run minimization
We will try to run minimization with categorical features and only a subset of the features with different possible values of target accuracy (how close to the original model's accuracy we want to get, 1 being same accuracy as for original data).

In [4]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

from apt.minimization import GeneralizeToRepresentative
from sklearn.model_selection import train_test_split

# default target_accuracy is 0.998
minimizer = GeneralizeToRepresentative(model, 
                                     categorical_features=categorical_features, features_to_minimize=QI)

# Fitting the minimizar can be done either on training or test data. Doing it with test data is better as the
# resulting accuracy on test data will be closer to the desired target accuracy (when working with training
# data it could result in a larger gap)
# Don't forget to leave a hold-out set for final validation!
X_generalizer_train, x_test, y_generalizer_train, y_test = train_test_split(x_test, y_test, stratify=y_test,
                                                                test_size = 0.4, random_state = 38)
X_generalizer_train.reset_index(drop=True, inplace=True)
y_generalizer_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
encoded_generalizer_train = preprocessor.transform(X_generalizer_train)
x_train_predictions = model.predict(encoded_generalizer_train)
minimizer.fit(X_generalizer_train, x_train_predictions, features_names=features)
transformed = minimizer.transform(x_test, features_names=features)

encoded_transformed = preprocessor.transform(transformed)
print('Accuracy on minimized data: ', model.score(encoded_transformed, y_test))

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): 0.805556
Improving accuracy
feature to remove: Credit_history
Removed feature: Credit_history, new relative accuracy: 0.819444
feature to remove: Other_installment_plans
Removed feature: Other_installment_plans, new relative accuracy: 0.847222
feature to remove: Duration_in_month
Removed feature: Duration_in_month, new relative accuracy: 0.847222
feature to remove: Property
Removed feature: Property, new relative accuracy: 0.847222
feature to remove: Housing
Removed feature: Housing, new relative accuracy: 0.847222
feature to remove: Purpose
Removed feature: Purpose, new relative accuracy: 0.986111
feature to remove: debtors
Removed feature: debtors, new relative accuracy: 0.986111
feature to remove: Job
Removed feature: Job, new relative accuracy: 1.000000
Accuracy on minimized data:  0.6666666666666666


#### Let's see what features were generalized

In [5]:
generalizations = minimizer.generalizations
print(generalizations)

{'ranges': {}, 'categories': {}, 'untouched': ['Foreign_worker', 'Other_installment_plans', 'Existing_checking_account', 'Purpose', 'debtors', 'Housing', 'N_people_being_liable_provide_maintenance', 'Present_employment_since', 'Installment_rate', 'Credit_history', 'Property', 'Present_residence', 'Age', 'Credit_amount', 'Duration_in_month', 'Job', 'Personal_status_sex', 'Number_of_existing_credits', 'Savings_account', 'Telephone']}


We can see that for the default target accuracy of 0.998 of the original accuracy, no generalizations are possible (all features are left untouched, i.e., not generalized).

Let's change to a slightly lower target accuracy.

In [6]:
# We allow a 1% deviation in accuracy from the original model accuracy
minimizer2 = GeneralizeToRepresentative(model, target_accuracy=0.92, 
                                     categorical_features=categorical_features, features_to_minimize=QI)

minimizer2.fit(X_generalizer_train, x_train_predictions, features_names=features)
transformed2 = minimizer2.transform(x_test, features_names=features)

encoded_transformed2 = preprocessor.transform(transformed2)
print('Accuracy on minimized data: ', model.score(encoded_transformed2, y_test))
generalizations2 = minimizer2.generalizations
print(generalizations2)

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): 0.805556
Improving accuracy
feature to remove: Credit_history
Removed feature: Credit_history, new relative accuracy: 0.819444
feature to remove: Other_installment_plans
Removed feature: Other_installment_plans, new relative accuracy: 0.847222
feature to remove: Duration_in_month
Removed feature: Duration_in_month, new relative accuracy: 0.847222
feature to remove: Property
Removed feature: Property, new relative accuracy: 0.847222
feature to remove: Housing
Removed feature: Housing, new relative accuracy: 0.847222
feature to remove: Purpose
Removed feature: Purpose, new relative accuracy: 0.986111
Accuracy on minimized data:  0.6666666666666666
{'ranges': {}, 'categories': {'debtors': [['A103', 'A102'], ['A101']], 'Job': [['A173', 'A174'], ['A171'], ['A172']]}, 'untouched': ['Credit_amount', 'Duration_in_month', 'Credit_history', 'Foreign_

This time we were able to generalize two features (debtors and Job).